In [ ]:
pip install import-ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=7db02922a67848720d5417be591eb29e425558b57879ced0fb420a081730d30d
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import sys
sys.path.append('/content/gdrive/Shared drives/ECE765 Group Project')
%cd /content/gdrive/Shared drives/ECE765 Group Project

/content/gdrive/Shared drives/ECE765 Group Project


In [ ]:
# Switch to Tensorflow 1 for VAE model to work
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
# import libraries
import keras
import import_ipynb
import pandas as pd
import math
import joblib
from sklearn.model_selection import train_test_split
from IPython.display import display
from keras.optimizers import RMSprop, Adam, SGD
from keras.layers import Input, Dense, Lambda, Flatten, Reshape, BatchNormalization, Activation, Dropout
from keras.optimizers import RMSprop, Adam, SGD
from keras.models import Model
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from torch.utils.data import Dataset, DataLoader
import DataPrep_CNN
import torch
import numpy as np
import time


Using TensorFlow backend.


importing Jupyter notebook from DataPrep_CNN.ipynb
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#Build Training Dataset

class TrainingDataset(Dataset):
    def __init__(self):
      self.preimages = read_training_data()
      self.images = normalized_images(self.preimages)
      self.labels = np.array([0] * 4739 + [1] * 4738)

    def __len__(self):
      return len(self.images)

    def __getitem__(self, idx):
      img = self.images[idx]
      label = self.labels[idx]
      return img, label

def trainDataset():
  train_dataset = TrainingDataset()
  return train_dataset

#Build Validation Dataset

class ValDataset(Dataset):
    def __init__(self):
      self.preimages = read_val_data()
      self.images = normalized_images(self.preimages)
      self.labels = np.array([0] * 500 + [1] * 500)

    def __len__(self):
      return len(self.images)

    def __getitem__(self, idx):
      img = self.images[idx]
      label = self.labels[idx]
      return img, label

def valDataset():
  val_dataset = ValDataset()
  return val_dataset

In [ ]:
trainImages = torch.load('/content/gdrive/Shared drives/ECE765 Group Project/trainImages_vae.pt')
valImages = torch.load('/content/gdrive/Shared drives/ECE765 Group Project/valImages_vae.pt')
x = trainImages.images.numpy()
X_dataset = x[0:4738]
X_dataset = np.moveaxis(X_dataset,1,3)
X_dataset.shape
X_train = X_dataset

In [ ]:
# load pca model
pca_model_file = '/content/gdrive/Shared drives/ECE765 Group Project/pca128_2000comp'
pca = joblib.load(pca_model_file)
n_components = pca.n_components_
print(n_components)

2000


In [ ]:
pca.n_features_

49152

In [ ]:
# cread data generator to load training data into the model
def data_generator(X_data, pca, batch_size):
    while True:
        index = np.arange(len(X_data))
        np.random.shuffle(index)
        for i in range(len(X_data) // batch_size):
            x_batch = pca.transform(X_data[index[i * batch_size: (i + 1) * batch_size]].reshape(batch_size, -1) / 255.)
            yield x_batch, x_batch


In [ ]:
# VAE parameters
img_rows, img_cols, img_chns = 128, 128, 3

if K.image_data_format() == 'channels_first':
    original_img_size = (img_chns, img_rows, img_cols)
else:
    original_img_size = (img_rows, img_cols, img_chns)

batch_size = 64
latent_dim = 512
intermediate_dim = 2048
epsilon_std = 1.0
epochs = 100
activation = 'relu'
dropout = 0.2
var_epsilon = 0.010
learning_rate = 0.0001

In [ ]:
# encoder network
x = Input(batch_shape=(batch_size,) + (n_components,))

batch_1 = BatchNormalization()(x)
hidden1_dense = Dense(intermediate_dim)(batch_1)
hidden1_batch = BatchNormalization()(hidden1_dense)
hidden1 = Activation(activation)(hidden1_batch)
dropout_1 = Dropout(dropout)(hidden1)

hidden2_dense = Dense(intermediate_dim)(dropout_1)
hidden2_batch = BatchNormalization()(hidden2_dense)
hidden2 = Activation(activation)(hidden2_batch)
dropout_2 = Dropout(dropout)(hidden2)

hidden3_dense = Dense(intermediate_dim)(dropout_2)
hidden3_batch = BatchNormalization()(hidden3_dense)
hidden3 = Activation(activation)(hidden3_batch)
dropout_3 = Dropout(dropout)(hidden3)

z_mean_1_dense = Dense(intermediate_dim)(dropout_3)
z_mean_1_batch = BatchNormalization()(z_mean_1_dense)
z_mean_1 = Activation(activation)(z_mean_1_batch)
z_mean_dropout_1 = Dropout(dropout)(z_mean_1)

z_mean_2_dense = Dense(intermediate_dim)(z_mean_dropout_1)
z_mean_2_batch = BatchNormalization()(z_mean_2_dense)
z_mean_2 = Activation(activation)(z_mean_2_batch)
z_mean_dropout_2 = Dropout(dropout)(z_mean_2)
z_mean = Dense(latent_dim)(z_mean_dropout_2)

z_log_var_1_dense = Dense(intermediate_dim)(dropout_3)
z_log_var_1_batch = BatchNormalization()(z_log_var_1_dense)
z_log_var_1 = Activation(activation)(z_log_var_1_batch)
z_log_var_dropout_1 = Dropout(dropout)(z_log_var_1)

z_log_var_2_dense = Dense(intermediate_dim)(z_log_var_dropout_1)
z_log_var_2_batch = BatchNormalization()(z_log_var_2_dense)
z_log_var_2 = Activation(activation)(z_log_var_2_batch)
z_log_var_dropout_2 = Dropout(dropout)(z_log_var_2)
z_log_var = Dense(latent_dim)(z_log_var_dropout_2)

In [ ]:
def sampling(args, batch_size=batch_size, latent_dim=latent_dim, epsilon_std=epsilon_std):
    z_mean, z_log_var = args
    
    epsilon = K.random_normal(shape=(batch_size, latent_dim),
                              mean=0., stddev=epsilon_std)
    
    return z_mean + K.exp(z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

In [ ]:
# decoder network
decoder_1_dense = Dense(intermediate_dim)
decoder_1_batch = BatchNormalization()
decoder_1 = Activation(activation)
decoder_dropout_1 = Dropout(dropout)

decoder_2_dense = Dense(intermediate_dim)
decoder_2_batch = BatchNormalization()
decoder_2 = Activation(activation)
decoder_dropout_2 = Dropout(dropout)

decoder_3_dense = Dense(intermediate_dim)
decoder_3_batch = BatchNormalization()
decoder_3 = Activation(activation)
decoder_dropout_3 = Dropout(dropout)

decoder_4_dense = Dense(intermediate_dim)
decoder_4_batch = BatchNormalization()
decoder_4 = Activation(activation)
decoder_dropout_4 = Dropout(dropout)

decoder_5_dense = Dense(intermediate_dim)
decoder_5_batch = BatchNormalization()
decoder_5 = Activation(activation)
decoder_dropout_5 = Dropout(dropout)

x_decoded_mean = Dense(n_components)

decoder_6_dense = Dense(n_components)
decoder_6_batch = BatchNormalization()
decoder_6 = Activation(activation)
decoder_dropout_6 = Dropout(dropout)
x_decoded_var = Dense(n_components, activation='relu')

In [ ]:
_decoder_1 = decoder_dropout_1(decoder_1(decoder_1_batch(decoder_1_dense(z))))
_decoder_2 = decoder_dropout_2(decoder_2(decoder_2_batch(decoder_2_dense(_decoder_1))))
_decoder_3 = decoder_dropout_3(decoder_3(decoder_3_batch(decoder_3_dense(_decoder_2))))
_decoder_4 = decoder_dropout_4(decoder_4(decoder_4_batch(decoder_4_dense(_decoder_3))))
_decoder_5 = decoder_dropout_5(decoder_5(decoder_5_batch(decoder_5_dense(_decoder_4))))

_x_decoded_mean = x_decoded_mean(_decoder_5)

_decoder_6 = decoder_dropout_6(decoder_6(decoder_6_batch(decoder_6_dense(_decoder_5))))
_x_decoded_var = x_decoded_var(_decoder_6)
_output = _x_decoded_mean

In [ ]:
def kl_loss(x, x_decoded_mean):
    kl_loss = - 0.5 * K.sum(1. + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1) * 0.0001
   
    return K.mean(kl_loss)

def logx_loss(x, x_decoded_mean):
    loss = (  0.5 * math.log(2 * math.pi)
            + 0.5 * K.log(_x_decoded_var + var_epsilon)
            + 0.5 * K.square(x - x_decoded_mean) / (_x_decoded_var + var_epsilon))
    loss = K.sum(loss, axis=-1)
    return K.mean(loss)

def vae_loss(x, x_decoded_mean):
    return logx_loss(x, x_decoded_mean) + kl_loss(x, x_decoded_mean)

In [ ]:
vae = Model(x, _output)
optimizer = Adam(lr=learning_rate)
vae.compile(optimizer=optimizer, loss=vae_loss)
vae.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (64, 1000)           0                                            
__________________________________________________________________________________________________
batch_normalization_9 (BatchNor (64, 1000)           4000        input_2[0][0]                    
__________________________________________________________________________________________________
dense_10 (Dense)                (64, 2048)           2050048     batch_normalization_9[0][0]      
__________________________________________________________________________________________________
batch_normalization_10 (BatchNo (64, 2048)           8192        dense_10[0][0]                   
____________________________________________________________________________________________

In [ ]:
start = time.time()

checkpoint_path = "/content/gdrive/Shared drives/ECE765 Group Project/vae_model128_cp.ckpt"
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1, period=5)

history = vae.fit_generator(
    data_generator(X_train, pca, batch_size),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs,
    #validation_data=(X_test_pca, X_test_pca),
    callbacks=[cp_callback],
    #verbose=0
)

done = time.time()
elapsed = done - start
print("Elapsed: ", elapsed)
vae.save("/content/gdrive/Shared drives/ECE765 Group Project/vae_model128.hdf5")
df = pd.DataFrame(history.history)
df.to_csv('/content/gdrive/Shared drives/ECE765 Group Project/vae_model128_history.csv')


Epoch 1/100
74/74 [==============================] - 19s 263ms/step - loss: 13303.5817
Epoch 2/100
74/74 [==============================] - 16s 217ms/step - loss: 6121.2980
Epoch 3/100
74/74 [==============================] - 16s 216ms/step - loss: 2074871677.3320
Epoch 4/100
74/74 [==============================] - 16s 218ms/step - loss: 10759.8203
Epoch 5/100
74/74 [==============================] - 16s 214ms/step - loss: 2758.9656

Epoch 00005: saving model to /content/gdrive/Shared drives/ECE765 Group Project/vae_model128_cp.ckpt
Epoch 6/100
74/74 [==============================] - 14s 192ms/step - loss: 2368.3251
Epoch 7/100
74/74 [==============================] - 16s 211ms/step - loss: 2671.4332
Epoch 8/100
74/74 [==============================] - 15s 207ms/step - loss: 2081.0769
Epoch 9/100
74/74 [==============================] - 15s 206ms/step - loss: 2019.4082
Epoch 10/100
74/74 [==============================] - 15s 207ms/step - loss: 1889.9535

Epoch 00010: saving model t

In [ ]:
'''
# serialize model to JSON
model_json = vae.to_json()
with open("/content/gdrive/Shared drives/ECE765 Group Project/vae_model32", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
vae.save_weights("/content/gdrive/Shared drives/ECE765 Group Project/vae_model32.h5")
print("Saved model to disk")
'''

Saved model to disk


In [ ]:
vae.save("/content/gdrive/Shared drives/ECE765 Group Project/vae_model32.hdf5")